# December 04, 2023

https://adventofcode.com/2023/day/4

In [1]:
import re
#from collections import defaultdict

In [49]:
test_str = f'''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

test_str = test_str.split("\n")

In [132]:
tmp = dict()
tmp[1] = 12
tmp[3] = 100
tmp[2] = 11

for k,v in tmp.items():
    print(k,v)

1 12
3 100
2 11


In [169]:
class Map:
    def __init__( self, lines ):
        m = re.fullmatch("^(.*)-to-(.*) map:$", lines[0])
        self.input, self.output = m[1], m[2]
        
        self.ranges = []
        self.ranges = [ [int(x) for x in y.strip().split()] for y in lines[1:]]

    def map_input( self, x ):
        for r in self.ranges:
            diff = x - r[1]
            if diff >= 0 and diff < r[2]:
                return r[0] + diff
            
        return x
    
    def map_range( self, start, len ):

        matched_starts = list()
        matched_lens = dict()

        ranges = []
        for r in self.ranges:
            if (start + len <= r[1]) or (start >= r[1] + r[2]):
                continue

            intx_start = max( start, r[1] )
            intx_stop = min( start + len - 1, r[1] + r[2] - 1)

            # Add to the output
            ranges.append( [intx_start + r[0] - r[1], intx_stop - intx_start + 1] )

            # Keep track of which numbers were matched
            matched_starts.append( intx_start )
            matched_lens[intx_start] = intx_stop - intx_start + 1

        # Determine unmatched regions
        # we cycle through all the matched regions from high to low
        # and keep track of any subranges in our input range that were never matched
        matched_starts.sort()
        cur = start
        for s in matched_starts:
            if cur < s:
                ranges.append( [cur, s-cur] ) # mark the numbers between cur and s as unmatched
            cur = max(cur, s + matched_lens[s] ) # move current to the spot after this matched region if it's in this matched range
            
            if cur >= start + len: # no more numbers to check
                break

        if cur < start + len:
            ranges.append( [cur, start + len - cur] )
                
            

        return ranges





In [170]:
def parse_input( lines ):
    seeds = [int(x) for x in re.sub("seeds: ", "", lines[0]).strip().split()]
    
    lines = lines[2:]
    paragraph = []

    maps = dict()
    for line in lines:
        if line == "":
            next_map = Map(paragraph)
            maps[ next_map.input ] = next_map
            paragraph = []

        else:
            paragraph.append(line)

    # extra step to map the last paragraph
    next_map = Map(paragraph)
    maps[ next_map.input ] = next_map
    return seeds, maps

seeds, maps = parse_input(test_str)
test = {'seeds':seeds, 'maps':maps}

In [171]:

part2( test['seeds'], test['maps'] , True)

Mapping seed to soil
[79, 14]
     ->[[81, 14]]
[55, 13]
     ->[[57, 13]]
Mapping soil to fertilizer
[81, 14]
     ->[[81, 14]]
[57, 13]
     ->[[57, 13]]
Mapping fertilizer to water
[81, 14]
     ->[[81, 14]]
[57, 13]
     ->[[53, 4], [61, 9]]
Mapping water to light
[81, 14]
     ->[[74, 14]]
[53, 4]
     ->[[46, 4]]
[61, 9]
     ->[[54, 9]]
Mapping light to temperature
[74, 14]
     ->[[45, 11], [78, 3]]
[46, 4]
     ->[[82, 4]]
[54, 9]
     ->[[90, 9]]
Mapping temperature to humidity
[45, 11]
     ->[[46, 11]]
[78, 3]
     ->[[78, 3]]
[82, 4]
     ->[[82, 4]]
[90, 9]
     ->[[90, 9]]
Mapping humidity to location
[46, 11]
     ->[[60, 1], [46, 10]]
[78, 3]
     ->[[82, 3]]
[82, 4]
     ->[[86, 4]]
[90, 9]
     ->[[94, 3], [56, 4], [97, 2]]


[[60, 1], [46, 10], [82, 3], [86, 4], [94, 3], [56, 4], [97, 2]]

In [172]:
fn = "data/05.txt"
with open(fn, "r") as file:
    text = file.readlines()

text = [x[:-1] for x in text]
seeds, maps = parse_input(text)
puzz = {'seeds':seeds, 'maps':maps}


### Part 1

In [71]:
def find_location( seed, maps, verbose = False ):
    output = seed
    output_type = 'seed'
    if verbose:
        print("Mapping seed #", seed)
    while output_type != 'location':
        output = maps[output_type].map_input(output)
        output_type = maps[output_type].output
        if verbose:
            print("    ->", output_type, output)

    return output


In [72]:
for s in test['seeds']:
    find_location( s, test['maps'], True)

Mapping seed # 79
    -> soil 81
    -> fertilizer 81
    -> water 81
    -> light 74
    -> temperature 78
    -> humidity 78
    -> location 82
Mapping seed # 14
    -> soil 14
    -> fertilizer 53
    -> water 49
    -> light 42
    -> temperature 42
    -> humidity 43
    -> location 43
Mapping seed # 55
    -> soil 57
    -> fertilizer 57
    -> water 53
    -> light 46
    -> temperature 82
    -> humidity 82
    -> location 86
Mapping seed # 13
    -> soil 13
    -> fertilizer 52
    -> water 41
    -> light 34
    -> temperature 34
    -> humidity 35
    -> location 35


In [84]:
locs = [find_location(x, puzz['maps']) for x in puzz['seeds']]

In [85]:
min(locs)

621354867

### Part 2

In [175]:
def part2( seeds, maps, verbose = False ):
    output_ranges = [ [seeds[i], seeds[i+1]] for i in range(0, len(seeds), 2) ]
    output_type = 'seed'

    while output_type != 'location':
        print('Mapping', output_type, 'to', maps[output_type].output)
        new_ranges = []
        for rng in output_ranges:
            if verbose:
                print(rng)
            mapped = maps[output_type].map_range( rng[0], rng[1] )
            if verbose:
                print("     ->", end='')
                print( mapped )
            new_ranges += mapped

        output_ranges = new_ranges
        output_type = maps[output_type].output

    return output_ranges


part2(test['seeds'], test['maps'], True )

Mapping seed to soil
[79, 14]
     ->[[81, 14]]
[55, 13]
     ->[[57, 13]]
Mapping soil to fertilizer
[81, 14]
     ->[[81, 14]]
[57, 13]
     ->[[57, 13]]
Mapping fertilizer to water
[81, 14]
     ->[[81, 14]]
[57, 13]
     ->[[53, 4], [61, 9]]
Mapping water to light
[81, 14]
     ->[[74, 14]]
[53, 4]
     ->[[46, 4]]
[61, 9]
     ->[[54, 9]]
Mapping light to temperature
[74, 14]
     ->[[45, 11], [78, 3]]
[46, 4]
     ->[[82, 4]]
[54, 9]
     ->[[90, 9]]
Mapping temperature to humidity
[45, 11]
     ->[[46, 11]]
[78, 3]
     ->[[78, 3]]
[82, 4]
     ->[[82, 4]]
[90, 9]
     ->[[90, 9]]
Mapping humidity to location
[46, 11]
     ->[[60, 1], [46, 10]]
[78, 3]
     ->[[82, 3]]
[82, 4]
     ->[[86, 4]]
[90, 9]
     ->[[94, 3], [56, 4], [97, 2]]


[[60, 1], [46, 10], [82, 3], [86, 4], [94, 3], [56, 4], [97, 2]]

In [177]:
locations = part2( puzz['seeds'], puzz['maps'] , False)
min( [x[0] for x in locations] )

Mapping seed to soil
Mapping soil to fertilizer
Mapping fertilizer to water
Mapping water to light
Mapping light to temperature
Mapping temperature to humidity
Mapping humidity to location


15880236